In [9]:
import os, io
import json
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pyarrow.parquet as pq
import pandas as pd
import glob
import tqdm
from torchvision import transforms

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# set gpu id
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### Data Loading Class for GSM8K

In [10]:
class GSM8KDataset(Dataset):
    def __init__(self, split):
        root = "datasets/grade_school_math/data"
        file_path = os.path.join(root, f"{split}.jsonl")
        with open(file_path, 'r') as f:
            self.data = [json.loads(line) for line in f]
        print(f"Loaded {len(self.data)} examples.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "question": item["question"],
            "answer": item["answer"]
        }

def gsm8k_dataloader(batch_size=32, split="train"):
    dataset = GSM8KDataset(split)
    return DataLoader(dataset, batch_size=batch_size, shuffle=(split == "train"))

### Qwen2-1.5B Inference

In [11]:
import os, json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

logging.getLogger("transformers").setLevel(logging.ERROR)

In [12]:
def load_qwen_model():
    model_path = "checkpoints/Qwen2-1.5B"
    print(f"Attempting to load model from local path: {model_path}")
    
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(device)

    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        raise
    
    return model, tokenizer, device

def llm_inference():
    model, tokenizer, device = load_qwen_model()
    dataloader = gsm8k_dataloader(batch_size=32, split="test")
    total_samples = len(dataloader)
    
    results = []
    process_single_sample = False
    
    for i, batch in enumerate(tqdm.tqdm(dataloader)):        
        question = batch["question"][0]
        print(f"  Question: {question[:50]}...")
        
        prompt = f"Question: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        try:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=50, max_time=30,
                                         num_return_sequences=1, do_sample=True, 
                                         temperature=0.7, top_p=0.95,
                                         pad_token_id=tokenizer.eos_token_id)
            
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = generated_text.split("Answer:")[-1].strip()
            print(f"  Answer generated: {answer[:50]}...")
        except Exception as e:
            print(f"  Error generating answer: {str(e)}")
            answer = "Error: Failed to generate answer"
        
        results.append({
            "question": question,
            "generated answer": answer,
            "ground truth": batch["answer"][0]
        })
        
        torch.cuda.empty_cache()  
        
        if process_single_sample:
            break 
    
    return results

In [5]:
results = llm_inference()
print(f"Processed {len(results)} questions from GSM8K dataset")

# Save results to a json file
output_file = "results/llm_inference_results.json"
if not os.path.exists("results"):
    os.makedirs("results")

with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)
print(f"Results saved to {output_file}")

Attempting to load model from local path: checkpoints/Qwen2-1.5B
Loaded 1319 examples.


  0%|          | 0/42 [00:00<?, ?it/s]

  Question: Janet’s ducks lay 16 eggs per day. She eats three ...


  2%|▏         | 1/42 [00:02<01:28,  2.15s/it]

  Answer generated: ```python
def daily_earnings():
    """Janet’s duc...
  Question: John takes care of 10 dogs.  Each dog takes .5 hou...


...

  Answer generated: ```python
def telephone_calls_needed():
    """Jas...
  Question: Steve put together a puzzle that took 10 hours of ...


 98%|█████████▊| 41/42 [00:41<00:00,  1.02it/s]

  Answer generated: ```python
from sympy import *

# Define the variab...
  Question: Tom's restaurant gets 6 reservations a night.  The...


100%|██████████| 42/42 [00:42<00:00,  1.01s/it]

  Answer generated: ```python
# each reservation gets 2 meals
meals_pe...
Processed 42 questions from GSM8K dataset
Results saved to results/llm_inference_results.json


### Finetune

In [18]:
import transformers
import os
from transformers import AdamW, get_linear_schedule_with_warmup

In [19]:
def finetune_model():
    # 加载基础模型
    model, tokenizer, device = load_qwen_model()
    
    # 检查pad_token是否存在
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 自定义微调数据集类
    class FinetuneGSM8KDataset(Dataset):
        def __init__(self, split, tokenizer, max_length=512):
            root = "datasets/grade_school_math/data"
            file_path = os.path.join(root, f"{split}.jsonl")
            with open(file_path, 'r') as f:
                self.data = [json.loads(line) for line in f]
            self.tokenizer = tokenizer
            self.max_length = max_length

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            item = self.data[idx]
            question = item["question"]
            answer = item["answer"]
            
            # 构造模型输入格式
            prompt = f"Question: {question}\nAnswer: {answer}"
            inputs = self.tokenizer(
                prompt,
                max_length=self.max_length,
                padding=False,
                truncation=True,
                return_tensors=None
            )
            
            # 分离输入和标签（问答分离）
            input_ids = inputs["input_ids"]
            # 找到Answer:的位置来分割输入输出
            answer_prefix = f"\nAnswer: "
            prompt_text = f"Question: {question}{answer_prefix}"
            prompt_len = len(self.tokenizer.encode(prompt_text, add_special_tokens=False))
            
            # 设置标签（只计算答案部分的loss）
            labels = [-100]*prompt_len + input_ids[prompt_len:]
            
            return {
                "input_ids": input_ids,
                "attention_mask": [1]*len(input_ids),
                "labels": labels
            }

    # 动态填充collate函数
    def collate_fn(batch):
        pad_token_id = tokenizer.pad_token_id
        max_length = max(len(item["input_ids"]) for item in batch)
        
        padded_batch = {
            "input_ids": [],
            "attention_mask": [],
            "labels": []
        }
        
        for item in batch:
            pad_len = max_length - len(item["input_ids"])
            padded_batch["input_ids"].append(
                item["input_ids"] + [pad_token_id]*pad_len
            )
            padded_batch["attention_mask"].append(
                item["attention_mask"] + [0]*pad_len
            )
            padded_batch["labels"].append(
                item["labels"] + [-100]*pad_len
            )
        
        # 转换为tensor并移到设备
        for key in padded_batch:
            padded_batch[key] = torch.tensor(padded_batch[key], dtype=torch.long).to(device)
            
        return padded_batch

    # 创建数据加载器
    train_dataset = FinetuneGSM8KDataset("train", tokenizer)
    train_loader = DataLoader(
        train_dataset,
        batch_size=4,  # 根据GPU显存调整
        shuffle=True,
        collate_fn=collate_fn
    )

    # 训练配置
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    num_epochs = 3
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1*total_steps),
        num_training_steps=total_steps
    )

    # 训练循环
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        progress_bar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}")
        
        for batch in progress_bar:
            optimizer.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            
            epoch_loss += loss.item()
            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
        
        # 打印epoch统计信息
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

    # 保存微调后的模型
    finetuned_path = "checkpoints/Qwen2-1.5B-finetuned"
    model.save_pretrained(finetuned_path)
    tokenizer.save_pretrained(finetuned_path)
    print(f"Fine-tuned model saved to {finetuned_path}")

In [20]:
# 修改后的推理函数（加载微调后的模型）
def llm_inference_after_finetune():
    # 加载微调后的模型
    finetuned_path = "checkpoints/Qwen2-1.5B-finetuned"
    print(f"Loading fine-tuned model from {finetuned_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(finetuned_path, trust_remote_code=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(finetuned_path, trust_remote_code=True).to(device)
    
    dataloader = gsm8k_dataloader(batch_size=32, split="test")
    total_samples = len(dataloader)
    
    results = []
    process_single_sample = False
    
    for i, batch in enumerate(tqdm.tqdm(dataloader)):        
        question = batch["question"][0]
        print(f"  Question: {question[:50]}...")
        
        prompt = f"Question: {question}\nAnswer:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        
        try:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=50, max_time=30,
                                         num_return_sequences=1, do_sample=True, 
                                         temperature=0.7, top_p=0.95,
                                         pad_token_id=tokenizer.eos_token_id)
            
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            answer = generated_text.split("Answer:")[-1].strip()
            print(f"  Answer generated: {answer[:50]}...")
        except Exception as e:
            print(f"  Error generating answer: {str(e)}")
            answer = "Error: Failed to generate answer"
        
        results.append({
            "question": question,
            "generated answer": answer,
            "ground truth": batch["answer"][0]
        })
        
        torch.cuda.empty_cache()  
        
        if process_single_sample:
            break 
    
    return results

In [22]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 执行微调
finetune_model()

# 使用微调后的模型进行推理
results = llm_inference()

# 保存结果
output_file = "results/finetuned_llm_results.json"
with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)
print(f"Inference results saved to {output_file}")

Attempting to load model from local path: checkpoints/Qwen2-1.5B


Epoch 1: 100%|██████████| 1869/1869 [22:37<00:00,  1.38it/s, loss=0.2774]


Epoch 1 completed. Average Loss: 0.4077


Epoch 2: 100%|██████████| 1869/1869 [22:36<00:00,  1.38it/s, loss=0.0822]


Epoch 2 completed. Average Loss: 0.2379


...

  Answer generated: ```python
def calculate_calls():
    """Jason work...
  Question: Steve put together a puzzle that took 10 hours of ...


 98%|█████████▊| 41/42 [00:39<00:00,  1.04it/s]

  Answer generated: ```python
from sympy import *

# Define the variab...
  Question: Tom's restaurant gets 6 reservations a night.  The...


100%|██████████| 42/42 [00:40<00:00,  1.03it/s]

  Answer generated: ```python
# Tom's restaurant gets 6 reservations a...
Inference results saved to results/finetuned_llm_results.json
